In [1]:
!pip install biopython datasets transformers evaluate

In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
#from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import re
from tqdm.notebook import tqdm
from datasets import DatasetDict, Dataset
import datasets
import evaluate
import os
from pathlib import Path

2025-04-27 12:21:42.954224: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 12:21:42.956998: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 12:21:43.000169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-27 12:21:43.963029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
DATASET_LOCATION = Path("../datasets/bernett_v4/")

In [4]:
tokenizer = BertTokenizerFast.from_pretrained("Rostlab/prot_bert", do_lower_case=False, use_fast=True)
model = BertForSequenceClassification.from_pretrained("Rostlab/prot_bert", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Download and process Bernett's Dataset v4 (added Q96PU5)

Download the old v3 version, that is likely the one that ProtBERT-Gru-Attention used.

Training: Intra1, validation: intra0, test intra2

Each txt file has the protein ID pairs from the human proteome. The corresponding sequences are in the fasta file. It probably is better to pair the sequences and tokenize them before training.

In [10]:
train_pos = pd.read_csv(DATASET_LOCATION / "Intra1_pos_rr.txt", sep=" ", header=None)
train_neg = pd.read_csv(DATASET_LOCATION / "Intra1_neg_rr.txt", sep=" ", header=None)
val_pos = pd.read_csv(DATASET_LOCATION / "Intra0_pos_rr.txt", sep=" ", header=None)
val_neg = pd.read_csv(DATASET_LOCATION / "Intra0_neg_rr.txt", sep=" ", header=None)
test_pos = pd.read_csv(DATASET_LOCATION / "Intra2_pos_rr.txt", sep=" ", header=None)
test_neg = pd.read_csv(DATASET_LOCATION / "Intra2_neg_rr.txt", sep=" ", header=None)

print(f"Train size: {train_pos.shape[0]} positives, {train_neg.shape[0]} negatives, {train_pos.shape[0] + train_neg.shape[0]} total.")
print(f"Validation size: {val_pos.shape[0]} positives, {val_neg.shape[0]} negatives, {val_pos.shape[0] + val_neg.shape[0]} total.")
print(f"Test size: {test_pos.shape[0]} positives, {test_neg.shape[0]} negatives, {test_pos.shape[0] + test_neg.shape[0]} total.")

Train size: 81596 positives, 81596 negatives, 163192 total.
Validation size: 29630 positives, 29630 negatives, 59260 total.
Test size: 26024 positives, 26024 negatives, 52048 total.


Check for NaNs, correct shapes, etc

In [11]:
# Check shape and NaN values
dfs = [train_pos, train_neg, val_pos, val_neg, test_pos, test_neg]
results = {
    "shape_check": [df.shape[1] == 2 for df in dfs],
    "nan_check": [df.isna().sum().sum() == 0 for df in dfs]
}
results = pd.DataFrame(results, index=["train_pos", "train_neg", "val_pos", "val_neg", "test_pos", "test_neg"])
results

,shape_check,nan_check
train_pos,True,True
train_neg,True,True
val_pos,True,True
val_neg,True,True
test_pos,True,True
test_neg,True,True


Map to sequences and add the label:

In [12]:
from Bio import SeqIO

# Load your FASTA file and create a mapping of ID -> Sequence
seq_dict = {record.id: str(record.seq) for record in SeqIO.parse(DATASET_LOCATION / "human_swissprot_oneliner.fasta", "fasta")}

In [13]:
mapped_dfs = []
for i, df in enumerate(dfs):
  df = df.applymap(lambda id: seq_dict.get(id, np.nan))
  df.columns = ['seq1', 'seq2']
  df['label'] = (i+1) % 2
  mapped_dfs.append(df)

Now, we check if there is any nan sequences, since if there are, then they weren't in the fasta file:

In [14]:
results = {
    "shape_check": [df.shape[1] == 3 for df in mapped_dfs],
    "nan_check": [df.isna().sum().sum() for df in mapped_dfs]
}
results = pd.DataFrame(results, index=["train_pos", "train_neg", "val_pos", "val_neg", "test_pos", "test_neg"])
results

,shape_check,nan_check
train_pos,True,0
train_neg,True,0
val_pos,True,0
val_neg,True,0
test_pos,True,0
test_neg,True,0


In [15]:
mapped_dfs[0].head()

,seq1,seq2,label
0,MNHLNVLAKALYDNVAESPDELSFRKGDIMTVLEQDTQGLDGWWLC...,MSKPPPKPVKPGQVKVFRALYTFEPRTPDELYFEEGDIIYITDMSD...,1
1,MNTAPSRPSPTRRDPYGFGDSRDSRRDRSPIRGSPRREPRDGRNGR...,MQRRDDPAARMSRSSGRSGSMDPSGAHPSVRQTPSRQPPLPHRSRG...,1
2,MFRNQYDNDVTVWSPQGRIHQIEYAMEAVKQGSATVGLKSKTHAVL...,MAELQQLQEFEIPTGREALRGNHSALLRVADYCEDNYVQATDKRKA...,1
3,MERSGQRVTTWDCDQGKHSDSDYREDGMDLGSDAGSSSSSSRASSQ...,MYEGKKTKNMFLTRALEKILADKEVKKAHHSQLRKACEVALEEIKA...,1
4,MDEQSVESIAEVFRCFICMEKLRDARLCPHCSKLCCFSCIRRWLTE...,MHLARLVGSCSLLLLLGALSGWAASDDPIEKVIEGINRGLSNAERE...,1


Now there aren't any missing sequences!!

Finally, convert them to huggingface datasets:

In [16]:
dataset = DatasetDict({
  "train": Dataset.from_pandas(pd.concat([mapped_dfs[0], mapped_dfs[1]]).reset_index(drop=True)),
  "validation": Dataset.from_pandas(pd.concat([mapped_dfs[2], mapped_dfs[3]]).reset_index(drop=True)),
  "test": Dataset.from_pandas(pd.concat([mapped_dfs[4], mapped_dfs[5]]).reset_index(drop=True))
})
dataset

DatasetDict({
    train: Dataset({
        features: ['seq1', 'seq2', 'label'],
        num_rows: 163192
    })
    validation: Dataset({
        features: ['seq1', 'seq2', 'label'],
        num_rows: 59260
    })
    test: Dataset({
        features: ['seq1', 'seq2', 'label'],
        num_rows: 52048
    })
})

In [13]:
# Free some memory
del mapped_dfs, dfs, results, seq_dict, train_pos, train_neg, val_pos, val_neg, test_pos, test_neg

# Tokenizing

In [17]:
# from concurrent.futures import ProcessPoolExecutor
import math
from functools import partial

def tokenize(batch, tokenizer, N=500):
  seqs1, seqs2 = batch['seq1'], batch['seq2'] # batch must be a dataframe with two columns.

  # replace unknown aminoacids and turn to uppercase
  seqs1 = [re.sub(r"[UZOB]", "X", seq.upper()) for seq in seqs1]
  seqs2 = [re.sub(r"[UZOB]", "X", seq.upper()) for seq in seqs2]

  # Truncate sequences according to the paper
  seqs1 = [seq[:math.ceil(N/2)] + seq[-N//2:] if len(seq) > N else seq for seq in seqs1]
  seqs2 = [seq[:math.ceil(N/2)] + seq[-N//2:] if len(seq) > N else seq for seq in seqs2]

  # introduce spaces between sequences and pad those that are not at least 500
  seqs1 = [' '.join(seq) + f" {tokenizer.pad_token}"*(N - len(seq)) for seq in seqs1]
  seqs2 = [' '.join(seq) + f" {tokenizer.pad_token}"*(N - len(seq)) for seq in seqs2]

  return tokenizer(seqs1, seqs2, padding=False, truncation=False)

tokenize_batch = partial(tokenize, tokenizer=tokenizer, N=500)

In [18]:
tok_dataset = dataset.map(tokenize_batch, num_proc=1, batched=True, batch_size=32)

Map:   0%|          | 0/163192 [00:00<?, ? examples/s]

Map:   0%|          | 0/59260 [00:00<?, ? examples/s]

Map:   0%|          | 0/52048 [00:00<?, ? examples/s]

In [20]:
tok_dataset['train'] = tok_dataset['train'].shuffle(seed=12)

In [21]:
tok_dataset['train']['label'][:10]

[0, 1, 1, 0, 1, 0, 1, 0, 1, 0]

In [22]:
tok_dataset.save_to_disk("tok_dataset_v1")

Saving the dataset (0/3 shards):   0%|          | 0/163192 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/59260 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/52048 [00:00<?, ? examples/s]

# Fine-tune on PPI

The original paper says that they finetuned ProtBERT using 90% of each dataset. I assume the fine tuning task was to classify PPIs. I also assume that they left out of the training, the validation and test fractions.

In [18]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
model.requires_grad_(False)
model.bert.pooler.requires_grad_(True)
model.classifier.requires_grad_(True)
model.bert.encoder.layer[-1].requires_grad_(True)

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=1024, out_features=1024, bias=True)
      (key): Linear(in_features=1024, out_features=1024, bias=True)
      (value): Linear(in_features=1024, out_features=1024, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=1024, out_features=1024, bias=True)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=1024, out_features=4096, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=4096, out_features=1024, bias=True)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
)

In [19]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

13647874

In [ ]:
training_args = TrainingArguments(
    output_dir="ProtBERT_FT-PPI",
    #eval_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
    logging_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_dataset['train'],
    eval_dataset=tok_dataset['validation'],
    compute_metrics=compute_metrics,
    #tokenizer=tokenizer, #Not sure if it will pad the sequences unnecesarily
)
trainer.train()

Epoch,Training Loss,Validation Loss
